# Router


![Screenshot 2024-08-21 at 9.24.09 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac6543c3d4df239a4ed1_router1.png)


In [4]:
from langchain_ollama.chat_models import ChatOllama

def Multiply(a: int, b: int): 
    """Multiply call this tool 
    when you want to multiply two numbers 
    Args: 
        a:int 
        b:int
    Returns: 
        a*b
    """
    return a * b

llm_with_tools = ChatOllama(model="llama3-groq-tool-use",tempreture=0).bind_tools([Multiply])
llm_with_tools.invoke("multiply 5 and 5").tool_calls

[{'name': 'Multiply',
  'args': {'a': 5, 'b': 5},
  'id': 'a116b055-bcde-41a6-82e0-968b8d030535',
  'type': 'tool_call'}]

In [5]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

def tool_selector(state: MessagesState): 
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

#create builder
builder = StateGraph(MessagesState)

#adding nodes
builder.add_node("tool_selector", tool_selector)
builder.add_node("tools",ToolNode([Multiply]))

#adding edges
builder.add_edge(START,"tool_selector")
builder.add_conditional_edges("tool_selector", tools_condition)
builder.add_edge("tools",END)